In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import vapeplot 
from scipy import interp
import scipy.stats
import warnings
warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
from datetime import datetime
def timestamp(): return datetime.today().strftime('%Y%m%d')

from sklearn.preprocessing import QuantileTransformer
def quantile_scale(df,feats):
    qua = df
    scaler = QuantileTransformer(
        n_quantiles=10,
        random_state=42,
        ignore_implicit_zeros=True, #sparse matrix
    )
    # fit the scaler
    scaler.fit(qua[feats])
    # transform values
    qua[feats] = scaler.transform(qua[feats])
    return qua
def tiered(classes):
    trans = []
    for x in classes:
        if x==1: c=0
        if x==2: c=1
        if x>=3 and x<=4: c=2
        if x>=5 and x<=7: c=3
        if x>=8: c=4
        trans.append(c)
    return trans

In [3]:
EPI=7
merge_col = ['season','baker','index','episode','place']
tech = pd.read_csv("../../DATA/s11/deepbake_s11_technical_features.20201106.tsv",sep="\t")
star = pd.read_csv("../../DATA/s11/deepbake_judge_features_s11_e7.20201106.tsv",sep="\t")
gbbo = pd.merge(tech, star,  how='left', left_on=merge_col, right_on =merge_col)
gbbo = gbbo[['season','baker','episode','place','tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']]
gbbo.to_csv("deepbake_features_s11_e{}.tsv".format(EPI),sep="\t",index=False)
gbbo.head()

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad
0,11,Dave,1,0,0.0,0,0.00,0.0,0.00,0.0,0.00,0.0
1,11,Dave,2,0,0.0,0,0.50,1.0,0.50,1.0,0.00,0.0
2,11,Dave,3,0,0.0,0,0.33,0.0,0.33,0.0,0.33,1.0
3,11,Dave,4,0,0.0,0,0.25,0.0,0.25,0.0,0.25,0.0
4,11,Dave,5,0,0.0,0,0.20,0.0,0.40,1.0,0.20,0.0


In [4]:
gbbo = pd.read_csv("deepbake_features_s11_e{}.tsv".format(EPI),sep="\t")
feats = ['tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']
max_epi = EPI
gbbo = gbbo.loc[gbbo['episode']==max_epi]
gbbo = quantile_scale(gbbo,feats)
gbbo.head()

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad
6,11,Dave,7,0,0.000000,1.000000,1.0,0.0,0.888889,0.0,0.000000,0.0
14,11,Hermine,7,0,0.701080,0.544444,1.0,1.0,0.888889,1.0,0.000000,0.0
22,11,Laura,7,0,0.892768,0.818182,1.0,0.0,0.444444,0.0,0.777778,1.0
30,11,Linda,7,0,0.626062,0.000000,0.0,0.0,0.000000,0.0,1.000000,1.0
38,11,Loriea,7,1,0.090909,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0


In [5]:
tech = pd.read_csv("../../RESULTS/deepbake_features.20201016.tsv",sep='\t')
tech = tech.loc[tech['episode']==max_epi]
qua = quantile_scale(tech,feats)
qua['place']=tiered(qua['place'])
X, y = np.matrix(qua[feats]), np.array(qua['place'])

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.activations import relu

def create_model( nl1=1, nl2=1,  nl3=1, 
                 nn1=1000, nn2=500, nn3 = 200, lr=0.01, decay=0., l1=0.01, l2=0.01,
                act = 'relu', dropout=0,input_shape=None,output_shape=None):    
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture
    https://www.kaggle.com/arrogantlymodest/randomised-cv-search-over-keras-neural-network
    '''
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999,  decay=decay)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
    model = Sequential()
    first=True  
    for i in range(nl1):
        if first:
            model.add(Dense(nn1, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn1, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))    
    for i in range(nl2):
        if first:
            model.add(Dense(nn2, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn2, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))    
    for i in range(nl3):
        if first:
            model.add(Dense(nn3, input_dim=input_shape, activation=act, kernel_regularizer=reg))
            first=False
        else: 
            model.add(Dense(nn3, activation=act, kernel_regularizer=reg))
        if dropout!=0:
            model.add(Dropout(dropout))       
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model
##################################
feats = ['tech_mean','tech','mean_star','star','mean_good','good','mean_bad','bad']
tech = pd.read_csv("../../RESULTS/deepbake_features.20201016.tsv",sep='\t')
tech['place']=tiered(tech['place'])
input_shape = len(feats)
output_shape = len(set(tech['place']))


l1 = 0.0001
l2 = 0.0001
lr = 0.0001
nl1 = 1
nl2 = 1
nl3 = 1
nn1 = 800
nn2 = 800
nn3 = 300

dropout = 0.1
decay = 1e-09
act='relu'
n_dims = len(feats)
n_classes = len(set(tech['place']))


BATCH,EPOCHS = 12, 25

In [7]:
GBBO = pd.read_csv("deepbake_features_s11_e{}.tsv".format(EPI),sep="\t")
S11 = pd.DataFrame()
GBBO = GBBO.loc[GBBO['episode']<=EPI]
for e in set(GBBO['episode']):
    gbbo = GBBO.loc[GBBO['episode']==e]
    gbbo = quantile_scale(gbbo,feats)
    test = np.matrix(gbbo[feats])

    tech = pd.read_csv("../../RESULTS/deepbake_features.20201016.tsv",sep='\t')
    tech = tech.loc[tech['episode']==e]
    qua = quantile_scale(tech,feats)
    qua['place']=tiered(qua['place'])
    
    X, y = np.matrix(qua[feats]), np.array(qua['place'])
    
    nn = create_model( nl1=nl1, nl2=nl2,  nl3=nl3, 
                     nn1=nn1, nn2=nn2, nn3 = nn3, 
                     lr=lr, decay=decay, l1=l1, l2=l2,
                     act = act, dropout=dropout,
                     input_shape=n_dims,
                     output_shape=n_classes)
    
    nn.fit(X,y,validation_split=0., batch_size=BATCH, epochs=EPOCHS,verbose=0)

    preds = nn.predict_classes(test)
    probs = nn.predict_proba(test)
    gbbo['preds']=preds
    # probability baker is a finalist
    top = probs[:,0]
    # probability baker is a finalist or a runner-up
    top3 = probs[:,0]+probs[:,1]
    # bottom tier (8th and below)
    bot = probs[:,-1]
    # 5th - 7th
    nextbot = probs[:,-2]
    third = probs[:,-3]

    gbbo['bottom']=np.round(bot*100,decimals=2)
    gbbo['finalist']=np.round(top*100,decimals=2) 
    gbbo['top3'] = np.round(top3*100,decimals=2)
    gbbo['fifthseventh'] = np.round(nextbot*100,decimals=2)
    gbbo['thirdforth'] = np.round(third*100,decimals=2)
    S11 = pd.concat([S11,gbbo])
    gbbo.to_csv("deepbake_s11.week{}_preditions.txt".format(e),sep="\t",index=False)
    print(e)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use `model.predict()` instead.
1
2
3
4
5
6
7


In [8]:
gbbo.sort_values(['top3'],ascending=False)

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad,preds,bottom,finalist,top3,fifthseventh,thirdforth
14,11,Hermine,7,0,0.701080,0.544444,1.0,1.0,0.888889,1.0,0.000000,0.0,1,3.200000,15.93,81.430000,3.890000,11.480000
6,11,Dave,7,0,0.000000,1.000000,1.0,0.0,0.888889,0.0,0.000000,0.0,1,6.330000,19.68,60.439999,8.010000,25.209999
62,11,Marc,7,0,1.000000,0.727273,1.0,0.0,1.000000,1.0,0.777778,0.0,1,5.020000,13.57,60.189999,9.980000,24.809999
78,11,Peter,7,0,0.481481,0.636364,1.0,0.0,0.611111,0.0,0.000000,0.0,1,7.000000,16.48,58.959999,11.430000,22.620001
46,11,Lottie,7,0,0.869375,0.909091,1.0,0.0,0.444444,0.0,0.000000,0.0,1,5.630000,12.53,55.639999,11.880000,26.850000
22,11,Laura,7,0,0.892768,0.818182,1.0,0.0,0.444444,0.0,0.777778,1.0,3,4.820000,4.80,28.610001,43.410000,23.170000
94,11,Sura,7,4,0.321905,0.000000,0.0,0.0,0.611111,0.0,0.000000,0.0,4,31.950001,10.08,25.330000,24.820000,17.889999
38,11,Loriea,7,1,0.090909,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,4,43.910000,8.04,17.030001,27.150000,11.900000
54,11,Mak,7,2,0.126110,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,4,43.230000,7.81,16.740000,28.150000,11.870000
70,11,Mark,7,0,0.365180,0.000000,1.0,0.0,0.611111,0.0,0.777778,0.0,4,42.040001,2.92,9.090000,41.990002,6.880000


In [11]:
gbbo.sort_values(['thirdforth'],ascending=False)

,season,baker,episode,place,tech_mean,tech,mean_star,star,mean_good,good,mean_bad,bad,preds,bottom,finalist,top3,fifthseventh,thirdforth
46,11,Lottie,7,0,0.869375,0.909091,1.0,0.0,0.444444,0.0,0.000000,0.0,1,5.630000,12.53,55.639999,11.880000,26.850000
6,11,Dave,7,0,0.000000,1.000000,1.0,0.0,0.888889,0.0,0.000000,0.0,1,6.330000,19.68,60.439999,8.010000,25.209999
62,11,Marc,7,0,1.000000,0.727273,1.0,0.0,1.000000,1.0,0.777778,0.0,1,5.020000,13.57,60.189999,9.980000,24.809999
22,11,Laura,7,0,0.892768,0.818182,1.0,0.0,0.444444,0.0,0.777778,1.0,3,4.820000,4.80,28.610001,43.410000,23.170000
78,11,Peter,7,0,0.481481,0.636364,1.0,0.0,0.611111,0.0,0.000000,0.0,1,7.000000,16.48,58.959999,11.430000,22.620001
94,11,Sura,7,4,0.321905,0.000000,0.0,0.0,0.611111,0.0,0.000000,0.0,4,31.950001,10.08,25.330000,24.820000,17.889999
38,11,Loriea,7,1,0.090909,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,4,43.910000,8.04,17.030001,27.150000,11.900000
54,11,Mak,7,2,0.126110,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,4,43.230000,7.81,16.740000,28.150000,11.870000
14,11,Hermine,7,0,0.701080,0.544444,1.0,1.0,0.888889,1.0,0.000000,0.0,1,3.200000,15.93,81.430000,3.890000,11.480000
70,11,Mark,7,0,0.365180,0.000000,1.0,0.0,0.611111,0.0,0.777778,0.0,4,42.040001,2.92,9.090000,41.990002,6.880000
